In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from math import pi
# from scipy.optimize import root

def Dipolar_matrix(k, e_dd, phi1):
    if k==1:
        F1=0.8
        F2=0.8
    elif k==0:
        F1=1
        F2=2
    else:
        y1 = ((1-k**2)**(0.5))
        F0=np.real(1/y1*np.log((1+y1)/(1-y1)))
        F1=1/4*(10/(1-k**2)-6/((1-k**2)**2)+3*(k**4)/((1-k**2)**2)*F0)
        #F2=-1/(1-k**2)*(4-6/(1-k**2)+3/(1-k**2)*(k**2)*F0)
    A1=1-e_dd+3/4*e_dd*F1*(1*np.cos(phi1)**2+3*np.sin(phi1)**2)
    A2=3/4*e_dd*F1*np.sin(2*phi1)
    B1=1-e_dd+3/4*e_dd*F1*(3*np.cos(phi1)**2+1*np.sin(phi1)**2)
    B2=3/4*e_dd*F1*np.sin(2*phi1)
    return A1, A2, B1, B2
    
def Dipolar_angle(k, gamma1):
    phi1=gamma1
    phi2=pi/2+gamma1
    return phi1, phi2

    
def plot_ellipse_XY(k=1.0, e_dd=0.2, phi_deg=0.0, theta_deg=0.0):
    
    phi = np.radians(phi_deg)  # convert angle to radians
    theta = np.radians(theta_deg)

    f = plt.figure(figsize=(8, 4))
    gs = f.add_gridspec(1, 2, width_ratios=[1, 1])

    ax1 = f.add_subplot(gs[0])
    ax2 = f.add_subplot(gs[1])
    x = np.linspace(-1, 1, 400)
    t0=np.linspace(0,2*pi,200)
    y_cont,x_cont=0*t0, 0*t0

    ax1.plot(np.cos(t0), np.sin(t0), 'r')
    ax1.plot(-np.sin(theta)*x, np.cos(theta)*x, 'b')   
  
    #Calculate and plot lines of zero deflection
    phi1,phi2=Dipolar_angle(k,theta)
    ax1.plot(2*x*np.cos(phi1), 2*x*np.sin(phi1), 'c--')
    ax1.plot(2*x*np.cos(phi2), 2*x*np.sin(phi2), 'c--')
    ax2.plot(2*x*np.cos(phi1), 2*x*np.sin(phi1), 'c--')
    ax2.plot(2*x*np.cos(phi2), 2*x*np.sin(phi2), 'c--')

    #Meshgrid of points inside ellipse
    xs = np.linspace(-2, 2, 12)
    ys = np.linspace(-2, 2, 12)
    X, Y = np.meshgrid(xs, ys)
    X_flat = X.flatten()
    Y_flat = Y.flatten()
    mask = X_flat**2 + (Y_flat)**2 <= 1**2
    points = np.vstack((X_flat[mask], Y_flat[mask])).T  
    u = np.cos(phi) * np.ones(len(points))
    v = np.sin(phi) * np.ones(len(points))
    u = u * np.sign(np.cos(theta)*points[:,0]+np.sin(theta)*points[:,1])
    v = v * np.sign(np.cos(theta)*points[:,0]+np.sin(theta)*points[:,1])
    ax1.scatter(points[:,0], points[:,1], color='green', zorder=5, label='Uniform lattice points')
    ax1.quiver(points[:,0], points[:,1], u, v, angles='xy', scale_units='xy', scale=4, color='orange')

    
    #Calculate and plot curve of the constant acceleration and variated magnetic field
    for i in range(len(t0)):
        A1,A2,B1,B2=Dipolar_matrix(k, e_dd, t0[i])
        Sc_f2=1/(A1*B1-A2*B2)
        x_cont[i] = Sc_f2*(np.cos(theta)*A1-np.sin(theta)*A2)
        y_cont[i] = Sc_f2*(np.sin(theta)*B1-np.cos(theta)*B2)     
    
    ax2.plot(0.5*np.cos(t0), 0.5*np.sin(t0), 'r--')
    ax2.plot(0.5*x_cont, 0.5*y_cont, 'g:')
    
    B_vec=np.array([np.cos(phi),np.sin(phi)])
    A_vec=-np.array([np.cos(theta),np.sin(theta)])
    A1,A2,B1,B2=Dipolar_matrix(k, e_dd, phi)
    Sc_f2=1/(A1*B1-A2*B2)
    A_eff_vec=np.array([Sc_f2*(np.cos(theta)*A1-np.sin(theta)*A2),Sc_f2*(np.sin(theta)*B1-np.cos(theta)*B2)])

    # Vectrors on second subplot
    ax2.quiver(0, 0, B_vec[0], B_vec[1], angles='xy', scale_units='xy', scale=2, color='purple')
    ax2.text(0, -0.15, 'B', color='purple', fontsize=10)
    ax2.quiver(0, 0, A_vec[0], A_vec[1], angles='xy', scale_units='xy', scale=2, color='red')
    ax2.text(-0.1, 0.1, 'a', color='red', fontsize=10)
    ax2.quiver(0, 0, A_eff_vec[0], A_eff_vec[1], angles='xy', scale_units='xy', scale=2, color='green')
    ax2.text(0.05, -0.1, r"$\nabla n$", color='green', fontsize=10)


    ax1.set_title('Dipole distribution')
    ax2.set_title('Density gradient')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.grid(True)
    ax1.set_xlim(-1.5,1.5)
    ax1.set_ylim(-1.5,1.5)
    #.axis('equal')
    ax2.set_xlim(-1,1)
    ax2.set_ylim(-1,1)
    plt.show()

# Interactive sliders
# interact(
#     plot_ellipse_XY,
#     k=(0, 3, 0.05),
#     e_dd=(0, 1, 0.05),
#     phi_deg=(0, 360, 3),
#     theta_deg=(0, 360, 3)
# );


interactive(children=(FloatSlider(value=1.0, description='k', max=3.0, step=0.05), FloatSlider(value=0.2, desc…